# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
from sql_queries import *
from etl import main as run_ETL

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()

In [3]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

In [4]:
run_ETL()

76 files found in data/song_data
1/76 files processed.
2/76 files processed.
3/76 files processed.
4/76 files processed.
5/76 files processed.
6/76 files processed.
7/76 files processed.
8/76 files processed.
9/76 files processed.
10/76 files processed.
11/76 files processed.
12/76 files processed.
13/76 files processed.
14/76 files processed.
15/76 files processed.
16/76 files processed.
17/76 files processed.
18/76 files processed.
19/76 files processed.
20/76 files processed.
21/76 files processed.
22/76 files processed.
23/76 files processed.
24/76 files processed.
25/76 files processed.
26/76 files processed.
27/76 files processed.
28/76 files processed.
29/76 files processed.
30/76 files processed.
31/76 files processed.
32/76 files processed.
33/76 files processed.
34/76 files processed.
35/76 files processed.
36/76 files processed.
37/76 files processed.
38/76 files processed.
39/76 files processed.
40/76 files processed.
41/76 files processed.
42/76 files processed.
43/76 file

# Process `song_data`
In this first part, you'll perform ETL on the first dataset, `song_data`, to create the `songs` and `artists` dimensional tables.

Let's perform ETL on a single song file and load a single record into each table to start.
- Use the `get_files` function provided above to get a list of all song JSON files in `data/song_data`
- Select the first song in this list
- Read the song file and view the data

In [5]:
song_files = get_files("data/song_data")

In [6]:
filepath = song_files[0]
print(filepath)

/home/workspace/data/song_data/A/A/A/TRAAAAW128F429D538.json


In [7]:
df = pd.read_json(filepath,typ='series')
#df.head()
print(df)

num_songs                            1
artist_id           ARD7TVE1187B99BFB1
artist_latitude                   None
artist_longitude                  None
artist_location        California - LA
artist_name                     Casual
song_id             SOMZWCG12A8C13C480
title                 I Didn't Mean To
duration                       218.932
year                                 0
dtype: object


## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [8]:
song_data = df[['song_id','title','artist_id','year','duration']]
print(tuple(song_data.values))
print(type(song_data.values[3]))

('SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179)
<class 'int'>


#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [9]:
cur.execute(song_table_insert, tuple(song_data))
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [10]:
artist_data = df[['artist_id','artist_name','artist_location','artist_latitude','artist_longitude']]
print(artist_data.values)

['ARD7TVE1187B99BFB1' 'Casual' 'California - LA' None None]


#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [11]:
cur.execute(artist_table_insert, artist_data.values)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [12]:
log_files = get_files("data/log_data")

In [13]:
filepath_log_files= log_files[1]
print(filepath_log_files)

/home/workspace/data/log_data/2018/11/2018-11-11-events.json


In [14]:
df2=pd.read_json(filepath_log_files,typ='series',lines = True)
print(df2)

0     {'artist': 'Frumpies', 'auth': 'Logged In', 'f...
1     {'artist': 'Kenny G with Peabo Bryson', 'auth'...
2     {'artist': 'Biffy Clyro', 'auth': 'Logged In',...
3     {'artist': None, 'auth': 'Logged In', 'firstNa...
4     {'artist': 'HIM', 'auth': 'Logged In', 'firstN...
5     {'artist': 'Matmos', 'auth': 'Logged In', 'fir...
6     {'artist': 'Gary Allan', 'auth': 'Logged In', ...
7     {'artist': 'Miracle Fortress', 'auth': 'Logged...
8     {'artist': 'Don Omar', 'auth': 'Logged In', 'f...
9     {'artist': 'Jay-Z', 'auth': 'Logged In', 'firs...
10    {'artist': 'Red Hot Chili Peppers', 'auth': 'L...
11    {'artist': None, 'auth': 'Logged In', 'firstNa...
12    {'artist': 'Flogging Molly', 'auth': 'Logged I...
13    {'artist': 'Reverend Horton Heat', 'auth': 'Lo...
14    {'artist': 'Sea Wolf', 'auth': 'Logged In', 'f...
15    {'artist': 'Jason Mraz & Colbie Caillat', 'aut...
16    {'artist': 'Jamie Lidell', 'auth': 'Logged In'...
17    {'artist': 'Feist', 'auth': 'Logged In', '

## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [15]:
dict_log_file= df2[0]
[y for x,y in dict_log_file.items() if x=='ts']

[1541903636796]

In [16]:
time_data = [y for x,y in dict_log_file.items() if x=='ts']
df=pd.DataFrame(np.array(time_data),columns=["time_stamp"])
print(time_data[0])
#time_pd=pd.Series(pd.date_range(time_data[0], periods=1, freq="s"))
time_pd2=pd.to_datetime(df.time_stamp)
print("check time /n",time_pd2)
month = time_pd2.dt.month
day = time_pd2.dt.day
year = time_pd2.dt.year
hour = time_pd2.dt.hour
minute= time_pd2.dt.minute
weekday= minute= time_pd2.dt.minute
week=time_pd2.dt.week


time_df=pd.DataFrame(np.array([minute,hour,day,week,month,year,weekday]).T,columns=['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'])

print(time_df['start_time'])
column_labels = ()

1541903636796
check time /n 0   1970-01-01 00:25:41.903636796
Name: time_stamp, dtype: datetime64[ns]
0    25
Name: start_time, dtype: int64


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [17]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [18]:
dict_log_file= df2[0]
print(np.__version__)
users_table_data= [[x,y] for x,y in dict_log_file.items() if np.any([x==y for y in ["userId","firstName","lastName","gender","level"]])]
#print(np.array(users_table_data)[:,1])
user_data_array=np.array(users_table_data)
print(np.array([user_data_array[:,0]]).shape)
#print(sel_data_log_file)
user_df_temp = pd.DataFrame(np.array([user_data_array[:,1]]),columns=list(np.array(user_data_array)[:,0]))
user_df=user_df_temp[["userId","firstName","lastName","gender","level"]]
user_df["userId"]=user_df['userId'].astype(int)
print(user_df)

1.12.1
(1, 5)
   userId firstName lastName gender level
0      69  Anabelle  Simpson      F  free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [19]:
for i, row in user_df.iterrows():
    print(row.values)
    cur.execute(user_table_insert, list(row))
    conn.commit()

[69 'Anabelle' 'Simpson' 'F' 'free']


Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [20]:
for row in df2:
    print(row['song'], row['artist'], row['length'])
    print(song_select)
    # get songid and artistid from song and artist tables
    song_selct="SELECT song_id,artist_id FROM song_table_create where title = %s AND duration = %s"
    cur.execute(song_selct, ("I Didn't Mean To",218.93179))
   
    #cur.execute(song_select, [(row['song'], row['artist'], row['length'])])
    results_song_select = cur.fetchone()
    print("results_song_select--->",results_song_select)
    print("checking aritst-->",row["artist"])
    print("artist_select--->",artist_select)
    
    cur.execute(artist_select, (str(row["artist"]),))
    results_artist_select = cur.fetchone()
    print("results_artist_select--->",results_artist_select)
    
    if results_song_select:
        songid, artistid = results_song_select
    else:
        songid, artistid = None, None
    
    if results_artist_select:
        artist_id, name, location = results_artist_select
    else:
        artist_id, name, location = None, None, None

    # insert songplay record 
    # get this informtion from other tables
    song_play_data=[row["ts"], row["userId"], row["level"], songid, artistid, row["sessionId"], location, row["userAgent"]]
    print("song_play_data--->",song_play_data)
    song_play_data_checked=[x if x is not '' else None for x in song_play_data]
    cur.execute(songplay_table_insert, song_play_data_checked)
    conn.commit()

Fuck Kitty Frumpies 134.47791
SELECT song_id, artist_id FROM song_table_create where title = %s and duration = %s
results_song_select---> ('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
checking aritst--> Frumpies
artist_select---> SELECT artist_id, name, location FROM artist_table_create where name = %s
results_artist_select---> None
song_play_data---> [1541903636796, '69', 'free', 'SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1', 455, None, '"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.125 Safari/537.36"']
By The Time This Night Is Over Kenny G with Peabo Bryson 264.75057
SELECT song_id, artist_id FROM song_table_create where title = %s and duration = %s
results_song_select---> ('SOMZWCG12A8C13C480', 'ARD7TVE1187B99BFB1')
checking aritst--> Kenny G with Peabo Bryson
artist_select---> SELECT artist_id, name, location FROM artist_table_create where name = %s
results_artist_select---> None
song_play_data---> [1541903770796, '69', 'free',

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [ ]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.